In [83]:
import pandas as pd
import numpy as np

# 데이터 가공 및 생성

In [84]:
###읽기
resp_2014 = pd.read_csv('./ehresp_2014.csv/ehresp_2014.csv')
resp_2015 = pd.read_csv('./ehresp_all/ehresp_2015.csv')
resp_2015.columns = map(str.lower, resp_2015.columns)
print(len(resp_2015.columns))
resp = pd.concat([resp_2014, resp_2015])
resp.reset_index(inplace=True, drop=True)
print(len(resp))
resp.head()

37
21838


,tucaseid,tulineno,eeincome1,erbmi,erhhch,erincome,erspemch,ertpreat,ertseat,ethgt,...,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,euwic,exincome1
0,20140101140007,1,-2,33.200001,1,-1,-1,30,2,0,...,1,2,1,-1,2,1,2,170,1,2
1,20140101140011,1,1,22.700001,3,1,-1,45,14,0,...,1,2,1,-1,1,2,2,128,2,0
2,20140101140028,1,2,49.400002,3,5,-1,60,0,0,...,-1,-1,2,2,-1,-1,-1,270,2,12
3,20140101140063,1,-2,-1.000000,3,-1,-1,0,0,0,...,2,2,1,1,2,6,-1,-2,2,2
4,20140101140168,1,2,31.000000,3,5,-1,65,0,0,...,1,2,1,2,1,1,2,210,1,0


In [85]:
###erbmi의 nan 값을 제거
bmi_nan_index = resp.loc[resp['erbmi']<0].index
resp_bmi = resp.drop(bmi_nan_index)
print(len(resp.index) == len(resp_bmi.index) + len(bmi_nan_index))
print(len(resp_bmi))
print(resp.isnull().sum().sum())

True
20708
0


In [86]:
###의미가 없거나 이해할 수 없는 column들을 날림.
# 날릴 변수
# tulineno : 어차피 다 1임.
# erhhch : CPS 파일 관련 데이터인데 해당 데이터가 없음.
# eeincome1, euincome2 : erincome으로 손실없이 합쳐지는 항목 임.
# erspemch : CPS 파일이 없음
# ethgt : flag 임.
# etwgt : flag 임.
# eufinlwgt : 알 수 없는 weight 값
# euinclvl : 물어본 년도임
# euwic : 모르는 프로그램
resp_bmi.drop(labels=['tulineno','erhhch','eeincome1','erspemch','ethgt','etwgt','eufinlwgt','euwic','exincome1','euincome2'], inplace=True, axis=1)
print(len(resp_bmi.columns))

27


In [87]:
###마이너스 값들을 모두 NaN으로 바꿈.
resp_bmi_final = resp_bmi[resp_bmi>0]
resp_bmi_final.head()

,tucaseid,erbmi,erincome,ertpreat,ertseat,eudietsoda,eudrink,eueat,euexercise,euexfreq,...,euhgt,euinclvl,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt
0,20140101140007,33.200001,NaN,30.0,2.0,NaN,2.0,1.0,2.0,NaN,...,60,5,1.0,2.0,1.0,NaN,2.0,1.0,2.0,170
1,20140101140011,22.700001,1.0,45.0,14.0,NaN,2.0,1.0,2.0,NaN,...,63,5,1.0,2.0,1.0,NaN,1.0,2.0,2.0,128
2,20140101140028,49.400002,5.0,60.0,NaN,NaN,1.0,2.0,2.0,NaN,...,62,5,NaN,NaN,2.0,2.0,NaN,NaN,NaN,270
4,20140101140168,31.000000,5.0,65.0,NaN,NaN,1.0,2.0,1.0,5.0,...,69,5,1.0,2.0,1.0,2.0,1.0,1.0,2.0,210
5,20140101140559,30.700001,1.0,20.0,10.0,1.0,1.0,1.0,1.0,2.0,...,71,5,NaN,NaN,2.0,1.0,NaN,NaN,NaN,220


In [88]:
# #bmi_class 추가(저체중, 정상, 과체중, 비만)
# resp_bmi_final["bmi_class"]=1
# resp_bmi_final.loc[((resp_bmi_final["erbmi"]>18.5) & (resp_bmi_final["erbmi"]<25)),"bmi_class"]=2
# resp_bmi_final.loc[((resp_bmi_final["erbmi"]>25) & (resp_bmi_final["erbmi"]<30)),"bmi_class"]=3
# resp_bmi_final.loc[(resp_bmi_final["erbmi"]>30),"bmi_class"]=4
# resp_bmi_final.head(100);

In [89]:
#bmi_class 추가 binary
resp_bmi_final["bmi_class"]=1
resp_bmi_final.loc[(resp_bmi_final["erbmi"]>30),"bmi_class"]=2
resp_bmi_final.head()

,tucaseid,erbmi,erincome,ertpreat,ertseat,eudietsoda,eudrink,eueat,euexercise,euexfreq,...,euinclvl,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,bmi_class
0,20140101140007,33.200001,NaN,30.0,2.0,NaN,2.0,1.0,2.0,NaN,...,5,1.0,2.0,1.0,NaN,2.0,1.0,2.0,170,2
1,20140101140011,22.700001,1.0,45.0,14.0,NaN,2.0,1.0,2.0,NaN,...,5,1.0,2.0,1.0,NaN,1.0,2.0,2.0,128,1
2,20140101140028,49.400002,5.0,60.0,NaN,NaN,1.0,2.0,2.0,NaN,...,5,NaN,NaN,2.0,2.0,NaN,NaN,NaN,270,2
4,20140101140168,31.000000,5.0,65.0,NaN,NaN,1.0,2.0,1.0,5.0,...,5,1.0,2.0,1.0,2.0,1.0,1.0,2.0,210,2
5,20140101140559,30.700001,1.0,20.0,10.0,1.0,1.0,1.0,1.0,2.0,...,5,NaN,NaN,2.0,1.0,NaN,NaN,NaN,220,2


In [90]:
#Train/test 를 돌릴 machine learning 데이터 생성 과정(resp_bmi_final_cp는 학습용 데이터 입니다.)
#이 데이터는 분석을 통해 얻은 핵심 feature의 null 값만 없앤다.
#핵심 feature의 하나라도 null이 있는 row 없애기
resp_bmi_final_cp = resp_bmi_final.copy()
for feature in ['eugenhth','euexercise','erincome','eusnap','eusoda']:
    resp_bmi_final_cp = resp_bmi_final_cp[~resp_bmi_final_cp[feature].isnull()]

In [91]:
### 갯수 확인(feature 확인용)
print(len(resp_bmi_final.loc[resp_bmi_final["bmi_class"]==1]))
print(len(resp_bmi_final.loc[(resp_bmi_final["bmi_class"]==2)]))
# print(len(resp_bmi_final.loc[(resp_bmi_final["bmi_class"]==3)]))
# print(len(resp_bmi_final.loc[(resp_bmi_final["bmi_class"]==4)]))
# print(len(resp_bmi_final_cp.loc[((resp_bmi_final_cp["erbmi"]>30))]))
print(len(resp_bmi_final))

14709
5999
20708


In [92]:
# ### 갯수 확인(학습용)
print(len(resp_bmi_final_cp.loc[resp_bmi_final_cp["bmi_class"]==1]))
print(len(resp_bmi_final_cp.loc[(resp_bmi_final_cp["bmi_class"]==2)]))
# print(len(resp_bmi_final_cp.loc[(resp_bmi_final_cp["bmi_class"]==3)]))
# print(len(resp_bmi_final_cp.loc[(resp_bmi_final_cp["bmi_class"]==4)]))
# # print(len(resp_bmi_final_cp_cp.loc[((resp_bmi_final_cp_cp["erbmi"]>30))]))
print(len(resp_bmi_final_cp))

9534
3941
13475


In [93]:
###resp_bmi_final 데이터는 feature를 뽑기 위한 데이터이므로 null을 없애지 않고 모든 column을 보존하였다.
cols = resp_bmi_final.columns
print(cols)

Index(['tucaseid', 'erbmi', 'erincome', 'ertpreat', 'ertseat', 'eudietsoda',
       'eudrink', 'eueat', 'euexercise', 'euexfreq', 'eufastfd', 'eufastfdfrq',
       'euffyday', 'eufdsit', 'eusnap', 'eugenhth', 'eugroshp', 'euhgt',
       'euinclvl', 'eumeat', 'eumilk', 'euprpmel', 'eusoda', 'eustores',
       'eustreason', 'eutherm', 'euwgt', 'bmi_class'],
      dtype='object')


In [94]:
resp_bmi_final.isnull().sum().sum()

89401

In [95]:
resp_bmi_final_cp.isnull().sum().sum()

48291

In [96]:
#학습용 데이터 생성
resp_bmi_final_old = resp_bmi_final.copy()
test_class_1 = resp_bmi_final_cp.loc[resp_bmi_final_cp["bmi_class"]==1].sample(n=190)
test_class_1;
resp_bmi_final_cp = resp_bmi_final_cp.drop(test_class_1.index)
print(len(resp_bmi_final_cp))
test_class_2 = resp_bmi_final_cp.loc[resp_bmi_final_cp["bmi_class"]==2].sample(n=190)

test_class_2;
resp_bmi_final_cp = resp_bmi_final_cp.drop(test_class_2.index)
print(len(resp_bmi_final_cp))

13285
13095


In [97]:
test_list = [test_class_1, test_class_2]
test = pd.concat(test_list)

In [98]:
len(test.loc[test['bmi_class']==1])

190

In [99]:
print(len(test))
print(test.isnull().sum().sum())
print(test[['eugenhth','euexercise','erincome','eusnap','eusoda']].isnull().sum().sum())

380
1422
0


In [100]:
train = resp_bmi_final_cp.copy()
print(train.isnull().sum().sum())
print(train[['eugenhth','euexercise','erincome','eusnap','eusoda']].isnull().sum().sum())

46869
0


In [101]:
len(train)

13095

In [102]:
train.head()

,tucaseid,erbmi,erincome,ertpreat,ertseat,eudietsoda,eudrink,eueat,euexercise,euexfreq,...,euinclvl,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,bmi_class
2,20140101140028,49.400002,5.0,60.0,NaN,NaN,1.0,2.0,2.0,NaN,...,5,NaN,NaN,2.0,2.0,NaN,NaN,NaN,270,2
4,20140101140168,31.000000,5.0,65.0,NaN,NaN,1.0,2.0,1.0,5.0,...,5,1.0,2.0,1.0,2.0,1.0,1.0,2.0,210,2
5,20140101140559,30.700001,1.0,20.0,10.0,1.0,1.0,1.0,1.0,2.0,...,5,NaN,NaN,2.0,1.0,NaN,NaN,NaN,220,2
6,20140101140610,33.299999,1.0,30.0,5.0,NaN,1.0,1.0,2.0,NaN,...,5,1.0,2.0,3.0,2.0,2.0,5.0,2.0,200,2
9,20140101140665,28.299999,1.0,80.0,NaN,2.0,1.0,2.0,2.0,NaN,...,5,1.0,2.0,1.0,1.0,3.0,1.0,2.0,170,1


In [103]:
train = train.sort_values('tucaseid')

In [104]:
train.head(20)

,tucaseid,erbmi,erincome,ertpreat,ertseat,eudietsoda,eudrink,eueat,euexercise,euexfreq,...,euinclvl,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,bmi_class
2,20140101140028,49.400002,5.0,60.0,NaN,NaN,1.0,2.0,2.0,NaN,...,5,NaN,NaN,2.0,2.0,NaN,NaN,NaN,270,2
4,20140101140168,31.000000,5.0,65.0,NaN,NaN,1.0,2.0,1.0,5.0,...,5,1.0,2.0,1.0,2.0,1.0,1.0,2.0,210,2
5,20140101140559,30.700001,1.0,20.0,10.0,1.0,1.0,1.0,1.0,2.0,...,5,NaN,NaN,2.0,1.0,NaN,NaN,NaN,220,2
6,20140101140610,33.299999,1.0,30.0,5.0,NaN,1.0,1.0,2.0,NaN,...,5,1.0,2.0,3.0,2.0,2.0,5.0,2.0,200,2
9,20140101140665,28.299999,1.0,80.0,NaN,2.0,1.0,2.0,2.0,NaN,...,5,1.0,2.0,1.0,1.0,3.0,1.0,2.0,170,1
10,20140101140685,40.500000,1.0,35.0,20.0,1.0,1.0,1.0,2.0,NaN,...,5,NaN,NaN,2.0,1.0,1.0,1.0,NaN,282,2
11,20140101140702,28.000000,5.0,NaN,5.0,NaN,1.0,1.0,2.0,NaN,...,5,1.0,2.0,1.0,2.0,1.0,2.0,2.0,125,1
12,20140101140725,27.900000,1.0,25.0,10.0,NaN,1.0,1.0,2.0,NaN,...,5,NaN,NaN,2.0,2.0,NaN,NaN,NaN,235,1
13,20140101140759,30.400000,5.0,150.0,5.0,1.0,1.0,1.0,2.0,NaN,...,5,NaN,NaN,2.0,1.0,NaN,NaN,NaN,200,2
16,20140101140836,25.799999,2.0,105.0,2.0,NaN,1.0,1.0,1.0,7.0,...,5,1.0,2.0,1.0,2.0,1.0,2.0,2.0,132,1


In [105]:
train.to_csv("food_train_1415_pure_selected.csv")
test.to_csv("food_test_balanced_pure_selected.csv")
resp_bmi_final.to_csv("food_binary.csv")

# 부록